In [ ]:
import numpy as np
import pandas as pd

# Intro to Pandas

# 1. Load data

We are using house sale price dataset that can be obtained from Kaggle: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/description

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, the competition challenges you to predict the final price of each home.

## Tasks:
1.1. Load `train.csv` file using `pd.read_csv()` function.

In [ ]:
path = !pwd
path = path[0]
path

In [ ]:
df_train = pd.read_csv(path+'/house-prices-advanced-regression-techniques/train.csv')

1.2. Print top 10 and last 10 observations in the table using `.head()` and `.tail()`

In [ ]:
df_train.head(10)

In [ ]:
df_train.tail(10)

1.3. Print all the data columns names using method `.columns`

In [ ]:
df_train.columns

1.4. Print number of rows and columns using method `.shape`

In [ ]:
df_train.shape

In [ ]:
pd.options.display.max_columns = 30
df_train.head(10)

# 2. Data exploration

## Tasks:
2.1. Use pandas `.describe()` to display basic statistic about the data.

In [ ]:
df_train.describe()

2.2. Use methods `.min()`, `.max()`, `.mean()`, `.std()` to display specific statistics about the data.

In [ ]:
df_train.mean()

2.3. Count number of unique values in every column `.nunique()`. What does this tells you about the features, which are most likely categorical and which are most likely numerical?

In [ ]:
df_train.nunique()

2.4. Use method `.count()` to count the number of non-NA cells in each column. Are there any missing values in the data? 
Missing values can be imputed with a mean value, dummy value or based on some other logic depending on the feature using `.fillna()` method.

In [ ]:
df_train.count()[df_train.count()<1460]

In [ ]:
df_train.GarageYrBlt.isna().sum()

2.5. Use method `.dtypes` field to display data types in columns. What are the columns with dtype int64?

In [ ]:
df_train.dtypes

In [ ]:
df_train.dtypes[df_train.dtypes == "int64"]

2.6. Use method `.value_counts()` to count number of unique values in a specific column.

In [ ]:
df_train.Heating.value_counts()

# 3. Data selection

In pandas.DataFrame you can select

1. Row/s by position (integer number \[0 .. number of rows - 1\]) `.iloc` or by DataFrame.index `.loc`:
```
data.loc[0]
data.loc[5:10]
data.iloc[0]
data.iloc[5:10]
```

2. Columns by name
```
data[columname]
```
3. Row/s and columns
```
data.loc[10, columname]
data.iloc[10, columname]
```
4. Using boolean mask
```
data[data[columname] > value]
```
You can combine multiple conditions using `&` or `|` (and, or)

```
cond1 = data[columname1] > value1
cond2 = data[columname2] > value2
data[cond1 & cond2]
```
5. Using queries `.query()`:
```
value = 5
data.query("columname > value")
```
You could combine multiple conditions using `and`, `or`

```
data.query("(columname1 > value1) and (columname2 > value2)")
```
and others. See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html for more examples.


## Tasks:

3.1. How many bedrooms does a house in row 7 have?

In [ ]:
df_train.iloc[7].BedroomAbvGr

3.2. How many houses has 3 kitchens?

In [ ]:
df_train[df_train.KitchenAbvGr==3].shape[0]

3.3. What is the percentage of houses built earlier than 1970?

In [ ]:
df_train[df_train.YearBuilt<1970].shape[0]/df_train.shape[0]

3.4. When was built the most expensive house?

In [ ]:
df_train[df_train.SalePrice==df_train.SalePrice.max()].YearBuilt

3.5. What roof style has a house built in 2005 with a central air conditioning and 11911 sqft lot size?

In [ ]:
df_train.CentralAir.unique()

In [ ]:
cond1 = df_train.CentralAir=='Y'
cond2 = df_train.YearBuilt==2005
cond3 = df_train.LotArea==11911
df_train[cond1 & cond2 & cond3].RoofStyle

3.6. What is the median lot size in the most popular zone?

In [ ]:
df_train[df_train.MSZoning==df_train.MSZoning.value_counts().idxmax()].LotArea.median()

# 4. Groupby
from the documentation https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

By “group by” we are referring to a process involving one or more of the following steps:

    - Splitting the data into groups based on some criteria.
    - Applying a function to each group independently.
    - Combining the results into a data structure.
    
---
`.groupby()` is one of the most powerfull tool for feature engineering. Very often it is used to group object with the same categorical characteristics and compute some statistics (e.g. mean, max, etc.) of a their numerical characteric. 

## Tasks
4.1. Compute mean remodel date (`YearRemodAdd`) for each overall condition (`OverallCond`)

In [ ]:
df_train.groupby('OverallCond')['YearRemodAdd'].mean()

4.2. Compute min and max price for each date (MM.YYYY)

In [ ]:
df_train.groupby(['YrSold', 'MoSold'])['SalePrice'].agg([min, max])

4.3. Create a new feature `StyleArea` for `df_train` indicating minimum above ground living area (`GrLivArea`) within the group of houses with specific `RoofStyle`, `Foundation`, `Heating`, and `GarageType`. 

In [ ]:
style = ['RoofStyle', 'Foundation', 'Heating', 'GarageType']
gp = df_train.groupby(style)['GrLivArea'].transform(min)
df_train['gp_feature'] = gp

In [ ]:
df_train['gp_feature']

In [ ]:
df_train['GrLivArea']

In [ ]:
df_train = df_train.drop(columns=['gp_feature'])

# 5. Data visualisation

In [ ]:
import seaborn as sns
sns.set(font_scale=1.2, style="whitegrid", palette='magma')
import matplotlib.pyplot as plt

## Tasks

5.1. Plot number of missing values as pandas `bar` plot

In [ ]:
#plot with pandas
missing = df_train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

5.2. Plot target variable distribution using `sns.distplot`

In [ ]:
sns.distplot(df_train['SalePrice'])

5.3. Visualise feature correlation matrix using `sns.heatmap`

In [ ]:
corr = df_train.corr()
fig = plt.figure(figsize=(12, 9))
sns.heatmap(corr, vmax=.8, square=True)

In [ ]:
#zoom in
k = 10 #number of variables for heatmap
cols = corr.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, 
                 fmt='.2f', annot_kws={'size': 10}, 
                 yticklabels=cols.values, xticklabels=cols.values)
plt.show()


5.4. Use `sns.boxplot` to show sale price variability within each OverallQual category

In [ ]:
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
fig = sns.boxplot(x=var, y="SalePrice", data=data)

5.5. Study relationship between price and GrLivArea feature (above grade (ground) living area square feet) using `scatter` plot

In [ ]:
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice')

5.6. Use `sns.pairplot` to visialise pairwise relations for 'SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', and 'YearBuilt'.

In [ ]:
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols])

5.7. Use `sns.FacetGrid` to create the following figure <img src="FacetGrid.png" width="600">

In [ ]:
g = sns.FacetGrid(df_train, col='FullBath', row='GarageCars',
                  margin_titles=True, hue='OverallQual')

g.map_dataframe(sns.scatterplot, x="YearRemodAdd", y="SalePrice", s=50, alpha=0.6)
g.set_axis_labels("YearRemodAdd", "SalePrice")
g.add_legend()
g.fig.set_size_inches(8,10)
g.tight_layout()
